# formata-texto
Esse script recebe arquivos .txt extraídos do site da Câmara dos Deputados e executa operações de limpeza e formatação usando regex.

#### Importação de pacotes

In [1]:
import glob
import os
import pandas as pd
import re
from unidecode import unidecode

#### Ler arquivos externos

In [2]:
def find_fpaths(dir_path, pattern):
    '''
    >> DESCRIÇÃO
    
    Usa o módulo glob para buscar todos os arquivos
    que correspondam ao padrão passado na variável 
    pattern'. Retorna uma lista de paths no formato 
    string. 
    
    >> PARÂMETROS
    
    dir_path -> uma string com o caminho para o
    diretório onde a busca pelos arquivos será
    realizada.
    
    pattern -> uma string com o padrão de texto
    que deve ser procurado no diretório.
    
    '''
    
    full_pattern = dir_path + pattern
    files = glob.glob(full_pattern)
    
    return files

In [3]:
def make_df(file_list):
    '''
    >> DESCRIÇÃO
    
    Lê a lista de arquivos e configura o conteúdo
    em um dataframe com os seguintes campos:
    PRESIDENTE | CONTEUDO | ARQUIVO | ANO
    
    >> PARÂMETROS
    
    file_list -> uma lista de filepaths em formato
    string. Ela é gerada anteriormente, na função
    find_fpaths.
    
    '''
    
    # Lê o conteúdo dos arquivos de texto na lista
    content = [ open(file).read() for file in file_list ]
    file_list = [ unidecode(file) for file in file_list ]
    
        
    # Transforma eum dataframe
    df = pd.DataFrame({
        'FILE'     : [ file for file in file_list ],
        'CONTENT'    : [ unidecode(item) for item in content ],
        'SESSION_TYPE' : [ re.search("\-([A-Z\s]+)\-", file).group(1) for file in file_list ],
        'SESSION_DATE' : [ re.search("\d{8}", file).group(0) for file in file_list ]
    })
    
    df["SESSION_DATE"] = pd.to_datetime(df.SESSION_DATE, format = "%d%m%Y")

    return df

#### Funções de formatação e busca usando regex

In [4]:
def find_speakers(string):
    
    '''
    Essa função detecta o padrão de texto
    que antecede a fala de um deputado e
    retorna um objeto match (via re.find_all).
    Ele é útil para detectar QUANTOS deputados
    falaram em determinada string textual.
    '''
    
    pattern = "((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?"
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    return matches

In [29]:
def extract_full_quote(string):
    
    '''
    Essa função extrai todas as falas de Jair Bolsonaro
    em uma determinada string. O pattern de regex encontra,
    primeiro, uma fala qualquer de Jair Bolsonaro. Então, pega
    tudo que está entre essa fala e a fala de outro parlamentar 
    ou o fim do arquivo. Isso é necessário porque há arquivos
    que misturam a fala de vários parlamentares, geralmente
    quando estão envolvidos em uma discussão.
    '''
    
    if "O SR. PRESIDENTE (Jair Bolsonaro)" in string:
        pattern = "O SR\. PRESIDENTE (\(Jair Bolsonaro\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
        group_no = 1

    else:
        pattern  = "O?\s?SR\.?\s?(DEPUTADO)?\s+JAIR\s+BOLSONARO\s?(\((Bloco\/)?\w{2,}\s?\-\s?\w{2}[^)]+\))?(.*?)((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?)|$)"
        group_no = 3
        
    regexp = re.compile(pattern)
    matches = re.findall(regexp, string)
    
    full_quote = [ ]
    for match in matches:
        quote = match[group_no]
        quote = ' '.join(quote.split()) # Remove espaços múltiplos internos usando a operação join de lista
        quote = quote.replace("- ", "")
        full_quote.append(quote)
        
    full_quote = ' [ INTERRUPÇÃO ] '.join(full_quote)
    return full_quote

#### Funções para aplicar operações de regex no dataframe

In [30]:
def apply_count_speakers(row):
    
    '''
    Aplica, linha a linha, a função
    find_speakers(string)
    '''
    
    matches = find_speakers(row.CONTENT)
    speaker_count = len(matches)
    
    return pd.Series({ "SPEAKER_COUNT":speaker_count 
        })

In [31]:
def apply_extract_full_quote(row):
    
    '''
    Aplica, linha a linha, a função
    extract_full_quote(string)
    '''
    
    full_quote = extract_full_quote(row.CONTENT)
    return pd.Series({
        "FULL_QUOTE" : full_quote
    })

In [32]:
jb = make_df( find_fpaths("../data/txts/plenario/", "*.txt") )

In [33]:
jb["SPEAKER_COUNT"] = jb.apply(apply_count_speakers, axis=1)

In [34]:
jb["FULL_QUOTE"] = jb.apply(apply_extract_full_quote, axis=1)

BOLSO PRESIDENTE
BOLSO PRESIDENTE
BOLSO PRESIDENTE
BOLSO PRESIDENTE
BOLSO PRESIDENTE
BOLSO PRESIDENTE
BOLSO PRESIDENTE


In [35]:
jb[jb.FULL_QUOTE == ""]

,CONTENT,FILE,SESSION_DATE,SESSION_TYPE,SPEAKER_COUNT,FULL_QUOTE
73,Ata da 110a (centesima decima) Sessao da Camar...,../data/txts/plenario/10052017-9h30-HOMENAGEM-...,2017-05-10,HOMENAGEM,0,
105,Ata da 117a (centesima decima setima) Sessao d...,../data/txts/plenario/16052017-9h30-HOMENAGEM-...,2017-05-16,HOMENAGEM,0,
114,Ata da 376a (tricentesima septuagesima sexta) ...,../data/txts/plenario/19112013-10h00-HOMENAGEM...,2013-11-19,HOMENAGEM,0,
130,O SR. JAIR BOLSONARO,../data/txts/plenario/06022012-14h24-PEQUENO E...,2012-02-06,PEQUENO EXPEDIENTE,1,
131,Ata da 154a (centesima quinquagesima quarta) S...,../data/txts/plenario/06062013-10h00-HOMENAGEM...,2013-06-06,HOMENAGEM,0,
168,Ata da 123a (centesima vigesima terceira) Sess...,../data/txts/plenario/19052016-9h00-HOMENAGEM-...,2016-05-19,HOMENAGEM,0,
174,Ata da 60a (sexagesima) Sessao da Camara dos D...,../data/txts/plenario/30032016-9h00-HOMENAGEM-...,2016-03-30,HOMENAGEM,0,
305,Ata da 229a (ducentesima vigesima nona) Sessao...,../data/txts/plenario/15102014-9h30-HOMENAGEM-...,2014-10-15,HOMENAGEM,0,
317,Ata da 63a (sexagesima terceira) Sessao da Cam...,../data/txts/plenario/31032016-9h00-HOMENAGEM-...,2016-03-31,HOMENAGEM,0,
482,O SR. JAIR BOLSONARO,../data/txts/plenario/03062004-10h46-BREVES CO...,2004-06-03,BREVES COMUNICACOES,1,


In [36]:
jb[jb.CONTENT.str.contains("O SR. PRESIDENTE \(Jair Bolsonaro\)")]

,CONTENT,FILE,SESSION_DATE,SESSION_TYPE,SPEAKER_COUNT,FULL_QUOTE
302,O SR. PRESIDENTE (Jair Bolsonaro) - Quero regi...,../data/txts/plenario/21052015-14h12-PEQUENO E...,2015-05-21,PEQUENO EXPEDIENTE,1,"Quero registrar, com muita satisfacao e alegri..."
444,O SR. PRESIDENTE (Jair Bolsonaro) - Quero saud...,../data/txts/plenario/19092012-9h30-BREVES COM...,2012-09-19,BREVES COMUNICACOES,1,"Quero saudar o Deputado Marcelo Ortiz, nosso e..."
552,O SR. PRESIDENTE (Jair Bolsonaro) - Quero saud...,../data/txts/plenario/18042006-15h22-PEQUENO E...,2006-04-18,PEQUENO EXPEDIENTE,1,Quero saudar a Associacao de Subtenentes e Sar...
576,O SR. PRESIDENTE (Jair Bolsonaro) - Antes de d...,../data/txts/plenario/03062004-10h26-BREVES CO...,2004-06-03,BREVES COMUNICACOES,1,"Antes de dar prosseguimento a sessao, esta Mes..."
592,O SR. PRESIDENTE (Jair Bolsonaro) - Eu peco de...,../data/txts/plenario/02062015-9h56-BREVES COM...,2015-06-02,BREVES COMUNICACOES,1,Eu peco desculpas aos colegas. Estou mantendo ...
686,O SR. PRESIDENTE (Jair Bolsonaro) - Antes de d...,../data/txts/plenario/15042010-10h08-BREVES CO...,2010-04-15,BREVES COMUNICACOES,1,"Antes de dar prosseguimento a sessao, esta Mes..."
786,O SR. PRESIDENTE (Jair Bolsonaro) - Sras. e Sr...,../data/txts/plenario/14102010-10h21-HOMENAGEM...,2010-10-14,HOMENAGEM,1,"Sras. e Srs. Deputados, a Presidencia da Camar..."


In [41]:
jb.loc[686, 'CONTENT']

'O SR. PRESIDENTE (Jair Bolsonaro) - Antes de dar prosseguimento a sessao, esta Mesa da conhecimento ao Plenario do seguinteAto da PresidenciaNos termos do SS 2o'

In [ ]:
# TO DO

# Adicionar captura dos casos em que Jair Bolsonaro foi presidente da Câmara

In [40]:
for item in jb[jb.CONTENT.str.contains("O SR. PRESIDENTE \(Jair Bolsonaro\)")].FULL_QUOTE:
    print(item)
    print("***")
    print()

Quero registrar, com muita satisfacao e alegria, a presenca no plenario do companheiro do Rio de Janeiro, Deputado Estadual pelo PRB, Carlos Macedo, acompanhado do Deputado Roberto Sales. Seja bem-vindo a esta Casa! Esperamos ve-lo aqui em 2019.
***

Quero saudar o Deputado Marcelo Ortiz, nosso eterno Deputado e companheiro de bons momentos de futebol, de bons debates e que faz muita falta a esta Casa. Quero cumprimenta-lo pela visita e dizer da falta que faz ao nosso convivio. Se Deus quiser, em breve, o ouviremos aqui por ocasiao das eleicoes de 2014. Seja bem-vindo.
***

Quero saudar a Associacao de Subtenentes e Sargentos da Policia Militar de Sao Paulo. Sou Deputado Federal pelo Rio de Janeiro, mas sou do Vale do Ribeira, da cidade de Eldorado Paulista. Te-los aqui neste dia e um fato que muito me orgulha.
***

Antes de dar prosseguimento a sessao, esta Mesa da conhecimento ao Plenario do seguinteAto da Presidencia Nos termos do art. 38, combinado com o SS 1o do art. 33, todos do 